In [ ]:
from datasets import load_dataset
# news:
dataset = load_dataset("cnn_dailymail", "3.0.0")

[1]
8s
from datasets import load_dataset
# news:
dataset = load_dataset("cnn_dailymail", "3.0.0")
# dialogue summarization:
# dataset = load_dataset("samsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token
# dialogue summarization:
# dataset = load_dataset("samsum")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
max_input_length = 1024   # max length for articles
max_target_length = 128   # max length for summaries

In [ ]:
from transformers import AutoTokenizer

# Choose your model checkpoint
model_ckpt = "google/pegasus-cnn_dailymail"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Then define preprocess function
def preprocess_function(batch):
    # encode inputs
    inputs = tokenizer(batch["article"], max_length=max_input_length, truncation=True)

    # encode targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"], max_length=max_target_length, truncation=True)

    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply mapping
tokenized_ds = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

# Model checkpoint
model_ckpt = "google/pegasus-cnn_dailymail"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# pip install --upgrade transformers

In [ ]:
# pip install --upgrade transformers

In [ ]:
# pip install --upgrade transformers

In [ ]:
import transformers
print(transformers.__version__)  # should be >= 4.1

4.44.0


In [ ]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.44.0
/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
# !pip install --upgrade tranformers

ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers


In [ ]:
# !pip uninstall -y transformers
# !pip install transformers==4.44.0

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="outputs/pegasus",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    logging_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=8
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=007dc2aa5e50b5ca40fc2fbc7f5f177cd01a6872adf845b3e1de295097171b75
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate
rouge = evaluate.load("rouge")


In [ ]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # convert to percent
    result = {k: round(v * 100, 4) for k, v in result.items()}
    return result


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="outputs/pegasus",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=3,
    logging_dir="logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    predict_with_generate=True,
    report_to="none"  # 🚀 disables wandb and other trackers
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

article_text = """
The Pegasus model by Google is a state-of-the-art transformer architecture designed for abstractive text summarization.
It was trained on massive amounts of data and uses a novel pretraining objective called Gap Sentence Generation.
Pegasus has been shown to achieve state-of-the-art results on multiple summarization benchmarks.
"""

# Tokenize and summarize
inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=1024).to(device)
summary_ids = model.generate(
    **inputs,
    max_length=140,
    min_length=30,
    num_beams=4,
    length_penalty=2.0,
    early_stopping=True,
    no_repeat_ngram_size=3
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


Pegasus is a transformer architecture designed for abstractive text summarization.<n>It was trained on massive amounts of data and uses a novel pretraining objective called Gap Sentence Generation.


In [ ]:
# Save model & tokenizer locally
model.save_pretrained("pegasus_model")
tokenizer.save_pretrained("pegasus_model")

print("✅ Model saved in 'pegasus_model/'")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


✅ Model saved in 'pegasus_model/'


In [ ]:
!zip -r pegasus_model.zip pegasus_model


  adding: pegasus_model/ (stored 0%)
  adding: pegasus_model/special_tokens_map.json (deflated 82%)
  adding: pegasus_model/spiece.model (deflated 50%)
  adding: pegasus_model/tokenizer_config.json (deflated 94%)
  adding: pegasus_model/generation_config.json (deflated 45%)
  adding: pegasus_model/config.json (deflated 60%)
  adding: pegasus_model/tokenizer.json (deflated 78%)
  adding: pegasus_model/model.safetensors (deflated 7%)


In [ ]:
!zip -r pegasus_model.zip pegasus_model
from google.colab import files
files.download("pegasus_model.zip")


  adding: pegasus_model/ (stored 0%)
  adding: pegasus_model/special_tokens_map.json (deflated 82%)
  adding: pegasus_model/pegasus_model.zip (stored 0%)
  adding: pegasus_model/spiece.model (deflated 50%)
  adding: pegasus_model/tokenizer_config.json (deflated 94%)
  adding: pegasus_model/generation_config.json (deflated 45%)
  adding: pegasus_model/config.json (deflated 60%)
  adding: pegasus_model/tokenizer.json (deflated 78%)
  adding: pegasus_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm -f pegasus_model.zip
!zip -r pegasus_model.zip pegasus_model


  adding: pegasus_model/ (stored 0%)
  adding: pegasus_model/special_tokens_map.json (deflated 82%)
  adding: pegasus_model/pegasus_model.zip (stored 0%)
  adding: pegasus_model/spiece.model (deflated 50%)
  adding: pegasus_model/tokenizer_config.json (deflated 94%)
  adding: pegasus_model/generation_config.json (deflated 45%)
  adding: pegasus_model/config.json (deflated 60%)
  adding: pegasus_model/tokenizer.json (deflated 78%)
  adding: pegasus_model/model.safetensors (deflated 7%)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load model and tokenizer
model_ckpt = "google/pegasus-xsum"  # You can also use "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

def summarize_text(paragraph, max_length=60, min_length=10):
    # Tokenize input text
    inputs = tokenizer(paragraph, truncation=True, padding="longest", return_tensors="pt")

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,  # beam search for better summaries
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode and return summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
paragraph = """
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines
that are programmed to think and learn like humans. These machines can be trained to
perform tasks such as recognizing speech, making decisions, and translating languages.
AI is being increasingly used in various industries including healthcare, finance, and
transportation, and it is expected to revolutionize the way we live and work in the future.
"""

summary = summarize_text(paragraph)
print("Summary:", summary)


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Summary: The global artificial intelligence (AI) market is expected to grow at a Compound Annual Growth Rate (CAR) of more than 50% over the forecast period.


#### Streamlit Ui

In [ ]:
# !pip install streamlit pyngrok transformers


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("ngrok config add-authtoken 31BclgfQ51UNUPcWelfjtpaAUmp_5tioiPX6J9KbW6rq8NYww")  # paste your token here

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("31BclgfQ51UNUPcWelfjtpaAUmp_5tioiPX6J9KbW6rq8NYww")

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load model & tokenizer once
@st.cache_resource
def load_model():
    model_ckpt = "google/pegasus-xsum"
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return tokenizer, model, device

tokenizer, model, device = load_model()

def summarize_text(paragraph, max_length=60, min_length=10):
    inputs = tokenizer(paragraph, truncation=True, padding="longest", return_tensors="pt").to(device)
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

st.title("📄 Pegasus Text Summarizer")
st.write("Enter a paragraph and get a short, AI-generated summary.")

paragraph = st.text_area("Enter your text here:", height=200)
max_len = st.slider("Max Summary Length", 30, 200, 60)
min_len = st.slider("Min Summary Length", 10, 50, 15)

if st.button("Generate Summary"):
    if paragraph.strip():
        with st.spinner("Summarizing..."):
            summary = summarize_text(paragraph, max_length=max_len, min_length=min_len)
        st.subheader("Summary:")
        st.write(summary)
    else:
        st.warning("Please enter some text.")


Overwriting app.py


In [ ]:
# Run Streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Create public URL with ngrok
public_url = ngrok.connect(8501)
print("🌐 Streamlit App URL:", public_url)


🌐 Streamlit App URL: NgrokTunnel: "https://1da7fcb20658.ngrok-free.app" -> "http://localhost:8501"
